In [33]:
"""
Script para el procesamiento, limpieza e ingeniería de características de datos
de calidad del aire a partir de múltiples fuentes CSV.
"""

# 1. IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [34]:
# 2. CONFIGURACIÓN DE RUTAS Y PARÁMETROS
# Se recomienda usar Path para construir rutas de forma independiente al sistema operativo.
BASE_PATH = Path(r"B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML")
OUTPUT_PATH = BASE_PATH / "Resultados_Procesamiento" # Carpeta dedicada para salidas
GRAFICOS_LIMPIEZA_PATH = OUTPUT_PATH / "graficos_limpieza"

# Crear directorios de salida si no existen para evitar errores.
OUTPUT_PATH.mkdir(exist_ok=True)
GRAFICOS_LIMPIEZA_PATH.mkdir(exist_ok=True)

# Mapeo de nombres de archivo a nombres de columna simplificados para el DataFrame final.
FILE_MAP = {
    "datos_200101_250901cohrs.csv": "CO",
    "datos_200101_250901Humedahrs.csv": "Humedad",
    "datos_200101_250901no2hrs.csv": "NO2",
    "datos_200101_250901nohrs.csv": "NO",
    "datos_200101_250901noxhrs.csv": "NOx",
    "datos_200101_250901o3hrs.csv": "O3",
    "datos_200101_250901pm10hrs.csv": "PM10",
    "datos_200101_250901pm25hrs.csv": "PM25",
    "datos_200101_250901so2hrs.csv": "SO2",
    "datos_200101_250901temperaturahrs.csv": "Temp",
    "datos_200101_250901direccionvientohrs.csv": "Viento_Dir",
    "datos_200101_250901velocidadviento.csv": "Viento_Vel"
}

In [35]:
# 3. FUNCIONES AUXILIARES
def parse_date(row):
    """
    Convierte columnas de fecha (YYMMDD) y hora (HHMM) en un objeto datetime.
    Diseñada para ser aplicada fila por fila en un DataFrame.
    """
    try:
        # Asegura que los datos sean strings con el formato correcto.
        date_str = str(int(row.iloc[0])).zfill(6)
        time_str = str(int(row.iloc[1])).zfill(4)

        # El formato '2400' no es válido para datetime, se ajusta a '0000'.
        if time_str == '2400':
            time_str = '0000'

        # Combina y convierte, asumiendo siglo 21. 'coerce' devuelve NaT en caso de error.
        return pd.to_datetime(f"20{date_str} {time_str}", format='%Y%m%d %H%M', errors='coerce')
    except (ValueError, TypeError):
        return pd.NaT # Retorna 'Not a Time' si la conversión inicial falla.


def clasificar_calidad_aire(pm25):
    """Clasifica la calidad del aire en 5 categorías según la concentración de PM2.5."""
    if pd.isna(pm25):
        return np.nan # Propagar valores nulos
    if pm25 <= 12: return 0  # Muy Bueno
    if pm25 <= 35: return 1  # Bueno
    if pm25 <= 55: return 2  # Regular
    if pm25 <= 150: return 3 # Malo
    return 4                 # Muy Malo

In [36]:
# 4. CARGA Y UNIFICACIÓN DE DATOS
print("Iniciando carga y unificación de datos...")
df_final = pd.DataFrame()

for filename, col_name in FILE_MAP.items():
    path = BASE_PATH / filename
    try:
        df_temp = pd.read_csv(path, sep=';', on_bad_lines='skip', engine='python')

        # Identificar la columna de valor de forma robusta.
        val_col = next((c for c in df_temp.columns if "validado" in c.lower()), df_temp.columns[2])

        # Procesamiento del DataFrame temporal
        df_temp['Datetime'] = df_temp.apply(parse_date, axis=1)
        df_temp = df_temp.dropna(subset=['Datetime']) # Eliminar filas donde la fecha no se pudo parsear
        df_temp = df_temp.set_index('Datetime')

        # Limpiar y convertir la columna de valor
        df_temp[col_name] = pd.to_numeric(
            df_temp[val_col].astype(str).str.replace(',', '.'),
            errors='coerce'
        )

        # Unir al DataFrame principal
        if df_final.empty:
            df_final = df_temp[[col_name]]
        else:
            df_final = df_final.join(df_temp[[col_name]], how='outer')
        print(f"  - Procesado: {filename}")
    except Exception as e:
        print(f"Error procesando {filename}: {e}")

print("\nCarga completada. Iniciando limpieza de datos...")

Iniciando carga y unificación de datos...
  - Procesado: datos_200101_250901cohrs.csv
  - Procesado: datos_200101_250901Humedahrs.csv
  - Procesado: datos_200101_250901no2hrs.csv
  - Procesado: datos_200101_250901nohrs.csv
  - Procesado: datos_200101_250901noxhrs.csv
  - Procesado: datos_200101_250901o3hrs.csv
  - Procesado: datos_200101_250901pm10hrs.csv
  - Procesado: datos_200101_250901pm25hrs.csv
  - Procesado: datos_200101_250901so2hrs.csv
  - Procesado: datos_200101_250901temperaturahrs.csv
  - Procesado: datos_200101_250901direccionvientohrs.csv
  - Procesado: datos_200101_250901velocidadviento.csv

Carga completada. Iniciando limpieza de datos...


In [37]:
# 5. LIMPIEZA DE DATOS (INTERPOLACIÓN)
# Primero, interpolar basado en el tiempo. Es ideal para series temporales con huecos irregulares.
df_clean = df_final.interpolate(method='time')
# Luego, rellenar cualquier valor restante al principio o al final.
df_clean = df_clean.bfill().ffill()

print("Limpieza completada. Generando gráficos de verificación...")

Limpieza completada. Generando gráficos de verificación...


In [38]:
# 6. VISUALIZACIÓN DE LA LIMPIEZA
# Genera y guarda un gráfico por cada variable para comparar datos originales vs. limpios.
for col_name in df_clean.columns:
    plt.figure(figsize=(18, 6))

    # Usar una muestra para visualización clara (ej. 500 primeros puntos)
    sample_size = 500
    y_orig = df_final[col_name].iloc[:sample_size]
    y_clean = df_clean[col_name].iloc[:sample_size]

    plt.plot(y_clean.index, y_clean, color='blue', alpha=0.7, label='Datos Interpolados', linewidth=1.5)
    plt.plot(y_orig.index, y_orig, 'r.', markersize=4, label='Datos Originales')

    plt.title(f"Verificación de Limpieza de Datos: {col_name}", fontsize=16)
    plt.ylabel(col_name)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()

    # Guardar el gráfico en la carpeta designada
    plt.savefig(GRAFICOS_LIMPIEZA_PATH / f"{col_name}_limpieza.png", dpi=200)
    plt.close() # Cerrar la figura para liberar memoria

print(f"Gráficos de limpieza guardados en: '{GRAFICOS_LIMPIEZA_PATH}'")

Gráficos de limpieza guardados en: 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Resultados_Procesamiento\graficos_limpieza'


In [39]:
# 7. INGENIERÍA DE CARACTERÍSTICAS
print("\nIniciando ingeniería de características...")

# A. Transformar Dirección del Viento a componentes vectoriales
# Esto evita el problema de la discontinuidad (359° vs 0°).
wd_rad = np.deg2rad(df_clean['Viento_Dir'])
df_clean['Viento_Cos'] = np.cos(wd_rad)
df_clean['Viento_Sin'] = np.sin(wd_rad)
df_clean = df_clean.drop(columns=['Viento_Dir'])

# B. Crear la variable objetivo (Target) a partir de PM2.5
df_clean['Calidad_Aire'] = df_clean['PM25'].apply(clasificar_calidad_aire)

print("Ingeniería de características completada.")

# 8. ANÁLISIS Y VISUALIZACIÓN DEL TARGET
print("\nAnalizando distribución de la variable objetivo...")
conteo_clases = df_clean['Calidad_Aire'].value_counts().sort_index()
etiquetas = ['0: Muy Bueno', '1: Bueno', '2: Regular', '3: Malo', '4: Muy Malo']
colores = ['green', 'lime', 'yellow', 'orange', 'red']

plt.figure(figsize=(10, 6))
bars = plt.bar(etiquetas, conteo_clases, color=colores, edgecolor='black')
plt.title("Distribución de Clases de Calidad del Aire (Target)", fontsize=16)
plt.ylabel("Cantidad de Registros (Horas)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.bar_label(bars, padding=3)
plt.tight_layout()

# Guardar el gráfico de distribución
ruta_grafico_target = OUTPUT_PATH / "distribucion_clases_target.png"
plt.savefig(ruta_grafico_target, dpi=300)
plt.close()
print(f"Gráfico de distribución del target guardado en: '{ruta_grafico_target}'")



Iniciando ingeniería de características...
Ingeniería de características completada.

Analizando distribución de la variable objetivo...
Gráfico de distribución del target guardado en: 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Resultados_Procesamiento\distribucion_clases_target.png'


In [40]:
# 9. GUARDADO DEL DATASET FINAL
ruta_salida_csv = OUTPUT_PATH / "dataset_unificado_limpio.csv"
df_clean.to_csv(ruta_salida_csv)

print("\n[PROCESO COMPLETADO]")
print(f"Dataset procesado y guardado en: '{ruta_salida_csv}'")
print("\nPrimeras 5 filas del dataset final:")
print(df_clean.head())


[PROCESO COMPLETADO]
Dataset procesado y guardado en: 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Resultados_Procesamiento\dataset_unificado_limpio.csv'

Primeras 5 filas del dataset final:
                       CO  Humedad    NO2        NO       NOx   O3  PM10  \
Datetime                                                                   
2020-01-01 01:00:00  0.27  52.4383  18.89  12.99750  31.89010  2.0  66.0   
2020-01-01 02:00:00  0.18  58.2467  16.66   4.65388  21.30980  3.0  57.0   
2020-01-01 03:00:00  0.11  61.9592  11.98   1.12154  13.10050  4.0  42.0   
2020-01-01 04:00:00  0.10  63.6742   8.77   1.04120   9.81378  6.0  35.0   
2020-01-01 05:00:00  0.10  67.6276   6.87   1.00000   7.87377  6.0  36.0   

                     PM25   SO2     Temp  Viento_Vel  Viento_Cos  Viento_Sin  \
Datetime                                                                       
2020-01-01 01:00:00  24.0  1.06  20.0008    0.608334   -0.29435